# PyData 2019 Deep Learning Workshop Tutorial- Pipple & 510

This notebook runs you through all basic functionalities that Keras has to offer when building and training a Convolutional Neural Network (CNN). The notebook is configured to be used in Google Colaboratory: a free of use Jupyter Notebook environment running on Google servers.

Notebooks are documents which contain both computer code (e.g. python) and rich text elements (paragraph, equations, figures, links, etc…). Notebooks are both human-readable documents containing the analysis description and the results (figures, tables, etc..) as well as executable commands which can be run to perform data analysis. Google Colaboratory combines these features with other Google services such as Google Drive. 

Before continuing this tutorial make sure you select 'GPU' as hardware accelerator. This will speed up the time of training a CNN substantially. 

Runtime -> Change runtime type -> Hardware accelerator = 'GPU' -> Save

---



# Table of Contents


1.   Problem Description
2.   Retrieve data
3.   Convolutional Neural Networks (CNN)
4.   Image Data Generators
5.   Transfer Learning + Fine-Tuning a CNN (in Keras)
6.   Training a CNN (in Keras)
7.   Analysing Training Results
8.   Play Around!
9.   Challenge!



---
---
# 1. Problem Description


Every year, approximately 90 thousand people are killed and almost 160 million people are affected by natural disasters, such as earthquakes floods, wildfires, and droughts<sup>[1](#myfootnote1)</sup>. Early September 2017,  hurricane Irma struck the island of Sint Maarten, affecting 90% of the buildings and leaving 7000 people (roughly 17% of the total population) without a house. The effectiveness, efficiency, and swiftness of humanitarian aid in such situations depends on the availability of local information, such as maps. However, in practice this information is often not available. 

Initiatives like the MissingMaps<sup>[2](#myfootnote1)</sup> project help humanitarian aid organisations to obtain the necessary maps. Through MissingMaps, volunteers can remotely trace buildings, roads, and other
information in aerial imagery into OpenStreetMap<sup>[3](#myfootnote1)</sup>, an open source map data base. Subsequently, local volunteers can add more detailed information about the traced objects, e.g. street names and building characteristics. Currently, MissingMaps is the go-to solution for humanitarian aid organisations when up-to-date and complete map data is unavailable. However, the process of acquiring these maps is time consuming, labour intensive, and the quality depends heavily on the skills of the volunteers. 

As a result 510, the Netherlands Red Cross (NLRC) data team, is investigating the possibility of (partly) automating the mapping process of remote areas using aerial imagery and machine learning. The desired models should be able to detect (the outlines) of buildings within the aerial imagery, and determine additional building characteristics of each building. Both objectives have their own inherent challenges and data needs. Therefore, the project is split  into two: one project will focus on the automatic detection of buildings, and another on automatic classification of building characteristics in remotely sensed imagery. This tutorial will focus on the latter topic by classifying roof type characteristics of buildings, that could indicate the vulnerability of buildings to natural disasters. To be more specific, this tutorial will classify roof shapes, either flat or hipped, of individual buildings in Sint Maarten by the use of pre-trained Convolutional Neural Networks (CNNs), Python, Keras and Tensorflow. Data sets are obtained from aerial imagery and information from OpenStreetMap.


---
> <sup>[1](#myfootnote1)</sup> https://www.who.int/environmental_health_emergencies/natural_events/en/

> <sup>[2](#myfootnote1)</sup> https://www.missingmaps.org

> <sup>[3](#myfootnote1)</sup> https://www.openstreetmap.org



---
---

# 2. Retrieve Data

Retrieving images of roof shapes of individual building in Sint-Maarten can be done by running the cell below. Note that the images are divided in a flat and hipped labeled folder which are part of either the train or validation data set. It is important to properly structure the data before you start tackling an image recognition task. In this tutorial the problem will be tackled using a the high-level API Keras. Below figure illustrates a simplified structure of the data.


![alt text](https://cdn-images-1.medium.com/max/800/1*HpvpA9pBJXKxaPCl5tKnLg.jpeg)



In [0]:
!git clone https://github.com/PippleNL/pydata2019.git


import zipfile

for sets in ['train', 'validation']:

  # path to zip
  local_zip = f'pydata2019/{sets}.zip' 

  # extract zip file
  zip_ref = zipfile.ZipFile(local_zip, 'r')
  zip_ref.extractall('/tmp')
  zip_ref.close()

import os

base_dir = '/tmp/'

# data directories
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

The data sets are located in the /tmp/train and /tmp/validation folders, respectively. Some basic statistics are given below:

* The data is seperated into a training and validation set.
* The images belong to 2 classes; flat and hipped.
* The training set contains 3786 flat and 1467 hipped images.
* The validation set contains 811 flat images and 733 hipped images.

Below are some bash commands listed to guide you through the repository and let you get a feeling of how the data set is structured.

In [0]:
!ls '/tmp'

In [0]:
!ls '/tmp/train'

In [0]:
!ls '/tmp/validation'

In [0]:
!find '/tmp/validation/hipped' -type f -name "*.tif" | wc -l

---
---
## 3. Convolutional Neural Networks (CNNs)

Traditionally, image classiﬁcation consisted of three steps: (1) local image feature extraction, (2) feature coding, and (3) classiﬁcation. The ﬁrst step involves the use of predeﬁned feature extraction algorithms (e.g. SIFT) which extract local features from several patches within the image. Subsequently, the local image features are encoded by coding algorithms to generate more sparse features suitable for classiﬁcation. At last, the encoded image features are fed into a classiﬁer. 

In the past years image classiﬁcation problems have been dominated by CNN architectures, which signiﬁcantly outperform all previous state-of-the-art benchmarks<sup>[4](#myfootnote1)</sup>. CNN architectures are typically built out of convolutional layers, pooling layers, and fully connected layers. In these architectures, the convolutional layers and pooling layers are responsible for image feature extraction, and the fully connected layers for classiﬁcation.


![alt text](https://miro.medium.com/max/1569/1*XbuW8WuRrAY5pC4t-9DZAQ.jpeg)

The outstanding performance of CNNs in remote sensing is mainly due to the use of transfer learning: complex deep CNN architectures are pre-trained on enormous image databases (e.g. ImageNet<sup>[5](#myfootnote1)</sup>) and used in other image classiﬁcation problems. The idea is that pre-trained CNNs can generate useful features for almost any classification problem. Classifying these features, however, is different in all cases dependent on the classification problem and the target data set. Generally, there are three approaches when using existing CNN architectures on another data set: 
1. training the CNN from scratch using the target data set, 
2. ﬁne-tuning the CNN on the target data set, and 
3. using the CNN directly as feature extractor in combination with another classiﬁer. 

What approach achieves the best results depends on the characteristics of the target data set, and the data set the CNN was initially trained on. More information on CNNs and transfer learning for this specific use-case of classifying building characteristics can be found in Bart van Driel's Masters Thesis: Roof Type Classification in Aerial Imagery Using Convolution Neural Networks<sup>[6](#myfootnote1)</sup>.

In this tutorial we will guide you in how to fine-tune an existing CNN on the target data set by the use of TensorFlow and Keras (i.e. high-level API built on top of TensorFlow). 

---

><sup>[4](#myfootnote1)</sup> Krizhevsky et al., Imagenet classification with deep convolutional neural networks, 2012

><sup>[5](#myfootnote1)</sup> Deng et al., Imagenet: A large-scale hierarchical image database

><sup>[6](#myfootnote1)</sup> https://pure.tue.nl/ws/portalfiles/portal/125083941/Master_Thesis_Bart_van_Driel.pdf



---
---

## 4. Image Data Generators

When fine-tuning a CNN on the target data set, a CNN needs to be provided data (i.e. images). Loading large amounts of images into memory, however, quickly raises memory issues for most servers. Keras deals with this by introducing the so-called ImageDataGenerator objects.

ImageDataGenerator objects generate batches of tensor image data with real-time data augmentation. Data augmentation is used to enlarge the amount of train data and is proven to be an effective method to increase a model's performance. 

![alt text](https://nanonets.com/blog/content/images/2018/11/1_C8hNiOqur4OJyEZmC7OnzQ.png)

By doing this real-time only batches of data are loaded into memory such that training the model is much more efficient.

Next to loading images into memory, the ImageDataGenerator objects deal with reshaping the images to the desired input size. As you might imagine, it is important that all images of the train data set are reshaped to the size of the images on which the pre-trained CNN is trained. If not, the pre-trained CNN will not be able to extract the right deep features out of the images, which is necessary to classify the train set.

In the cells below one can see how the ImageDataGenerators are called and where these generator parameters can be adjusted to a customer's needs. Note that each pre-trained CNN has its own pre-processor function and target size.

For more information on Keras' ImageDataGenerators please visit: https://keras.io/preprocessing/image/#imagedatagenerator-class

In [0]:
%tensorflow_version 1.x
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input as preprocess_input_vgg16
from keras.applications.inception_v3 import preprocess_input as preprocess_input_inception
from keras.applications.xception import preprocess_input as preprocess_input_xception

seed = 42  # Make the Image Data Generator objects reproducible

def get_cnn_data_generators(cnn):
  """
  Function that returns Image Data Generator objects for the train and validation set related to a pre-trained cnn (i.e. VGG16, Inception, Xception)
  """
  # Different pre-trained CNN's have different images pre-processor functions
  if cnn == 'vgg16':
    pre_processor = preprocess_input_vgg16
  elif cnn == 'inception':
    pre_processor = preprocess_input_inception
  elif cnn == 'xception':
    pre_processor = preprocess_input_xception
  else:
    raise ValueError(f'Unknown pre-trained CNN. Got {cnn} whereas vgg16, inception or exception is expected.')


  # Below Image Data Generator object is related to the train data set
  # This generator is typically allowed to augment images such that the train data set can be enlarged
  train_datagen = ImageDataGenerator(
    preprocessing_function=pre_processor,  # Most of the pre-trained CNN's (i.e. transfer learning) come with a specific pre-processor function (i.e. required)
    rotation_range=180,  # The Image Data Generator object is allowed to rotate images up to 180 degrees (i.e. augmentation)
    horizontal_flip=True,  # The Image Data Generator is allowed to horizontally flip images (i.e. augmentation)
    vertical_flip=True,  # The Image Data Generator is allowed to vertically flip images (i.e. augmentation)
    # Type here your (extra) Image Data Generator specifications related to the train data set (i.e. augmentation)
    # For more information visit the link above
  )


  # Below Image Data Generator object is related to the validation set
  # This generator is typically NOT allowed to augment images to accurately measure your CNN's performance on real data!
  val_datagen = ImageDataGenerator(preprocessing_function=pre_processor)

  return train_datagen, val_datagen

To generate batches of real-time (augmented) data, Keras' ImageDataGenerators come with the so called flow_from_directory() method. This method directly links the images directories on the server's local file system to facilitate real-time batching. In this method one also specifies the target reshape size of the train and validation images.

In [0]:
def get_image_batches(train_datagen, val_datagen, cnn):
  """
  Takes the path to a directory & generates batches of (augmented) data reshaped to the desired input shape
  """
  # Different pre-trained CNN's use different target images sizes as input 
  if cnn == 'vgg16':
    target_size = (224, 224)  # All images will be resized to 224x224
  elif cnn == 'inception':
    target_size = (299, 299)  # All images will be resized to 299x99
  elif cnn == 'xception':
    target_size = (299, 299)  # All images will be resized to 299x299
  else:
    raise ValueError(f'Unknown pre-trained CNN. Got {cnn} whereas vgg16, inception or exception is expected.') 


  # Keras is able to directly augment and use images out of folders from the server's local file system using the flow_from_directory method
  # Below example uses the train Image Data Generator to pre-process and augment the images from the train_dir (on the server's local file system)
  # and resizes them to the CNN specific target input (e.g. the input resolution for the pre-trained VGG16 CNN)
  train_generator = train_datagen.flow_from_directory(
    train_dir,  # The train data set directory on the local file system
    target_size=target_size,  # All images will be resized to the CNN specific target input
    batch_size=16,  # Keras will send batches of 16 (augmented) train images to the CNN
    class_mode='sparse',  # Label array will be 1D integer labels 
    shuffle=True,  # Shuffling is typically used when generating a train data set 
    seed=seed)  # The seed used for reproducibility for shuffling and augmentation transformations


  # Below examples uses the validation Image Data Generator to pre-process the images from the validation_dir (on the server's local file system)
  # In general you want the same parameters settings as specified for the train data set
  validation_generator = val_datagen.flow_from_directory(
    validation_dir,  
    target_size=target_size, 
    batch_size=16,  
    class_mode='sparse',
    shuffle=False)  # However, shuffling is typically not desired for interpreting the results on the validation data set

  return train_generator, validation_generator

One way to call a batch of (augmented) images using ImageDataGenerator objects  is by method next(). By running the cells below one will get an idea of how such a train batch looks like, defined by the ImageDataGenerator's parameters.

In [0]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline


# retrieve Image Data Generator objects for the train set related to the Xception pre-trained CNN
pre_trained_cnn = 'xception'
train_data_gen, _ = get_cnn_data_generators(cnn=pre_trained_cnn)

# Generate batches of augmented train data using the flow_from_directory function in Keras
train_batch_gen, _ = get_image_batches(train_data_gen, _, cnn=pre_trained_cnn)

# return a batch of (augmented) train images and their labels
train_images, train_labels = train_batch_gen.next()

# plot the batch of (augmented) images
plt.figure(figsize=(25, 10))
for i, image in enumerate(train_images, 1):
  plt.subplot(4, 5, i)  
  plt.grid(False)
  plt.title(list(train_batch_gen.class_indices)[int(train_labels[i-1])])
  plt.axis('off')
  plt.imshow(image)


---
---
## 5.   Transfer Learning + Fine-Tuning a CNN (in Keras)

Transfer Learning basically includes loading a pre-trained CNN's parameters and its architecture (i.e. structure of layers). In this tutorial one gets the ablilty to choose between 3 different pre-trained CNNs:


*   VGG-16: Nework consisting of 23 layers in 5 blocks, where 2 or 3 convolutional layers are followed by a pooling layer. Has 138.357.544 trainable parameters and a size of 528 MB.
*   InceptionV3: Network consiting of 314 layers and is made up of 11 so-called inception blocks, making the network extremely deep but manageable in the number of trainable parameters. Has 23.851.784 trainable parameters and a size of 92 MB.
*   Xception: Network consiting of 134 layers and uses blocks similar to InceptionV3's inception blocks. Has 22.910.480 trainable parameters and a size of 88 MB.


For more information on these pre-trained CNNs, I would like to redirect you to Bart van Driel's Masters Thesis<sup>[6](#myfootnote1)</sup>. 


Fine-tuning a pre-trained CNN makes it usable for classifying roof shapes. This is done by removing all top classification layers of the pre-trained model and replacing these by newly built (with randomly assigned parameters) classification layers. These newly built classification layers are the ones that will be trained! The cell below defines a function that does this trick and builds a CNN model from an existing pre-trained CNN. By default only the newly added classification layers will be trained. One, however, could choose to re-train some of the pre-trained feature extraction layers to make them more relevant for the classification of roof types. In that case, the parameters will be trained starting from the parameter values that were subject to the train results of the imagenet classification problem. Feel free to experiment with any kind of newly built classification layers. More information on different classification layers and how to deploy them using keras, can be found in<sup>[7](#myfootnote1)</sup>.



---


><sup>[6](#myfootnote1)</sup> https://pure.tue.nl/ws/portalfiles/portal/125083941/Master_Thesis_Bart_van_Driel.pdf

> <sup>[7](#myfootnote1)</sup> https://keras.io/layers/core/

In [0]:
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.inception_v3 import InceptionV3

from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Input, Flatten
from keras.layers import Dropout, BatchNormalization

import warnings
warnings.filterwarnings("ignore")  # surpress library warnings


def get_cnn_model(cnn):
  """
  Retrieves the parameters and architecture of pre-trained CNNs (on ImageNet data) without the top (classification) layers.
  It builds on top of these retrieve (feature extraction) layers a block of layers used for classification. 
  """
  # Retrieve the pre-trained cnn on images of imagenet without the top-classification layers
  if cnn == 'vgg16':
    base_model  = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
  elif cnn == 'inception':
    base_model  = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
  elif cnn == 'xception':
    base_model  = Xception(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
  else:
    raise ValueError(f'Unknown pre-trained CNN. Got {cnn} whereas vgg16, inception or exception is expected.') 


  # Set all retrieved cnn (feature extraction) layers to non-trainable; adjust to own preference
  for layer in base_model.layers:
      layer.trainable = False


  # Add extra (trainable) classification layers on top of the last feature extraction layer that is part of the pre-trained cnn
  # Make sure that these classification layers are imported from the keras.layers module
  x = base_model.output  # Retrieve last feature extraction layer (i.e. non-trainable)
  x = Flatten()(x)  # Add flatten layer  (i.e. does not consist of parameters to be trained) 
  x = Dense(256, activation='relu', name='fc1')(x)  # Add the first non-pre-trained convolutional (classification) layer (i.e trainable)
  x = Dense(256, activation='relu', name='fc2')(x)  # Add the second non-pre-trained convolutional (classification) layer (i.e trainable)
  
  # Add the last non-pre-trained convolutional (classification) layer (i.e trainable) generating prediction output.
  preds = Dense(2, activation='softmax', name='preds')(x) # Make sure this layer has the same number of nodes as the number of classes in the classification problem


  # Build the model having non-trainable pre-trained feature extration layers and trainable classification layers
  model = Model(inputs=base_model.input, outputs=preds)

  return model 

---
---
## 6.   Training a CNN (in Keras)

Training a CNN that consists of pre-trained layers and newly built classification layers can be done by calling the Keras' fit_generator method <sup>[8](#myfootnote1)</sup>. This method trains the model on data generated batch-by-batch by an ImageDataGenereator object which facilitates real-time data augmentation. 

The fit_generator method is applied to a so-called compiled model, which basically states how Keras should train the model. By compiling the model, one configures the optimizer<sup>[9](#myfootnote1)</sup>, loss function<sup>[10](#myfootnote1)</sup> and evaluation metrics<sup>[11](#myfootnote1)</sup> that are used while training. 

The training results, that can be used to analyse the training progress or to compare models, are logged by so-called Keras callbacks<sup>[12](#myfootnote1)</sup>. In these callbacks one defines all metrics that need to be stored (and in which files) to make a fair comparison between training results of different models. 



---

><sup>[8](#myfootnote1)</sup> https://keras.io/models/sequential/

><sup>[9](#myfootnote1)</sup> https://keras.io/losses/

><sup>[10](#myfootnote1)</sup> https://keras.io/optimizers/

><sup>[11](#myfootnote1)</sup> https://keras.io/metrics/

><sup>[12](#myfootnote1)</sup> https://keras.io/callbacks/




The cell below defines a function in which a CNN model is compiled. Note that compiling can only be done after building a CNNs architecture!




In [0]:
from keras.optimizers import SGD


def build_compile_cnn(cnn):
  """
  Builds a CNN architecture based on one of the three pre-trained CNNs (on ImageNet data) and extra added classification layers.
  Compiles this built cnn by specifying a loss function, optimizer and evaluation metric.
  """

  # Retrieve feature extraction layers of the pre-trained CNN and add newly to be trained (classification) layers
  cnn_model = get_cnn_model(cnn)


  # Compile the model
  cnn_model.compile(loss='sparse_categorical_crossentropy',  # Depends on how the target labels (e.g. labelled images in the data sets) are defined; are integers in this tutorial
                    optimizer=SGD(),  # Stochastic gradient descent optimizer used to optimize parameters; note that when choosing a different optimizer it needs to be imported from the keras.optimizers library
                    metrics=['acc'])  # Typically one uses the model's accuracy
  
  return cnn_model

The cell below defines a function that initialises some Keras callbacks. To make it more understandable, one could say that there are 2 ways of storing training information (i.e. defining callbacks):

*   In seperate files: A callback writing training info in a (text) file. Examples are the loss function metric in each training iteration and the model's parameter values for the interation which achieved the best results (i.e. CSVLogger and ModelCheckPoint)
*   In a TensorBoard event folder: A callback that stores evaluation metrics that can interactively be accessed using TensorBoard software. TensorBoard will be used in this tutorial to analyse and compare training results of different models. By default a TensorBoard callback only saves metrics as accuracy and the loss for the train and validation set. If one wants to add metrics to this callback one could do this by following the code in the cell below. 


Moreover, make sure that each training run is stored in a unique file name or event folder. Otherwise, there is nothing to compare against.



In [0]:
from keras.callbacks import CSVLogger, ModelCheckpoint, TensorBoard
from os.path import join, exists
from keras import backend as K
from keras.callbacks import TensorBoard
import time
from os import makedirs


# Adding the learning rate of the training procedure to the TensorBoard event folder 
class LRTensorBoard(TensorBoard):
    def __init__(self, log_dir, **kwargs): 
        super().__init__(log_dir=log_dir, **kwargs)

    def on_epoch_end(self, epoch, logs=None):
        logs.update({'lr': K.eval(self.model.optimizer.lr)})  # Store learning rate as lr in the event folder
        super().on_epoch_end(epoch, logs)


def get_callbacks(model_name):
  """
  Instantiates different Keras Callbacks used to analyse and compare training results
  """
  # Define unique training ID such that logging events are stored in a unique folder
  model_id = time.strftime('%Y-%m-%d_%H-%M-%S')

  # Callback that streams epoch results to a csv file.
  callback_csv = CSVLogger(filename=join(base_dir, f'{model_name}_{model_id}.csv'), 
                          separator=',', 
                          append=False)  
  # Callback that saves the model after every epoch
  os.makedirs(join(base_dir, 'models'), exist_ok=True)  # create models directory if not already present
  callback_model = ModelCheckpoint(filepath=join(base_dir, 'models', f'{model_name}_{model_id}.hdf5'), 
                                  monitor='val_acc', verbose=1, 
                                  save_best_only=True, save_weights_only=True,  # both are set to TRUE to increase the training speed
                                  mode='auto', period=3)  
  # Callback that writes a log for TensorBoard, which allows you to visualize dynamic graphs of your training and test metrics
  # as well as activation histograms for the different layers in your model. 
  # (https://www.tensorflow.org/tensorboard/)
  callback_tensorboard = TensorBoard(log_dir=join(base_dir, 'logs', f'{model_name}_{model_id}'), 
                                     histogram_freq=0,
                                     write_graph=True,
                                     write_grads=True,
                                     batch_size=train_batch_gen.samples/train_batch_gen.batch_size,
                                     write_images=True)
   
  # Add Learning rate to tensorboard logging events
  callback_learning_rate = LRTensorBoard(log_dir=join(base_dir, 'logs', f'{model_name}_{model_id}'))

  return [callback_csv, callback_model, callback_tensorboard, callback_learning_rate]


The cell below calls the fit_generator method for the compiled CNN model. Note that in this cell all other previous defined functions are also called to create the necessary CNN architecture, configuration, data generators and callback objects. 

In [0]:
from sklearn.utils.class_weight import compute_class_weight

# Define the pre-trained CNN that will be used for transfer learning and classification of roof types
pre_trained_cnn = 'xception'  # 'vgg16', 'inception' or 'xception'


# Build and compile a to be trained cnn model (i.e. includes the model architecture, initial parameters, optimizer, loss and evaluation function)
cnn_model = build_compile_cnn(cnn=pre_trained_cnn)


# Create corresponding train and validation data generators that specify augmentation rules
train_data_gen, vali_data_gen = get_cnn_data_generators(cnn=pre_trained_cnn)


# Create generators to generate batches of augmented train data using the flow_from_directory function in Keras
train_batch_gen, vali_batch_gen = get_image_batches(train_data_gen, 
                                                    vali_data_gen, 
                                                    cnn=pre_trained_cnn)


# It might be interesting to add class weights as a parameter to the fit_generator method such that training can 'pay more attention' to samples from an under-represented class
# Uncomment below lines to calculate the class weights
# class_weights = compute_class_weight('balanced', 
#                                      np.unique(train_batch_gen.classes), 
#                                      train_batch_gen.classes)

# Create callbacks that log training results
callbacks = get_callbacks(model_name=pre_trained_cnn)


# Fit the CNN to training and validation data; add class_weights as a parameter when one wants to 'pay more attention' to samples from an under-represented class
trained_cnn = cnn_model.fit_generator(generator=train_batch_gen,  # ImageDataGenerator for the training data set
                                      steps_per_epoch=15,  # Total number of steps to yield from generator use steps_per_epoch=train_batch_gen.samples/train_batch_gen.batch_size to use a full trainingset per epoch
                                      epochs=3,  # Number of epochs to train the model. An epoch is an iteration over the entire data provided, as defined by steps_per_epoch
                                      verbose=1,  # 0, 1, or 2. Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch
                                      callbacks=callbacks,  # List of callbacks to apply during training
                                      validation_data=vali_batch_gen,  # ImageDataGenerator for the validation data set
                                      validation_steps=vali_batch_gen.samples/vali_batch_gen.batch_size,  # Total number of steps to yield from validation_data
                                      )



---
---

## 7.   Analysing Training Results

Analysing training results is done using TensorBoard in this tutorial. TensorBoard is TensorFlow's visualisation toolkit to track and visualise performance metrics that are stored via the earlier defined callbacks. 

Tensorboard visualises different training event logs. Consequently, when training multiple models, make sure that all events are stored in a unique folder. One typically uses TensorBoard's visualisation to determine the epoch number after which training results become stale or how different learning rates influence the level of (optimization) convergence.

By running the cell below one starts an interactive TensorBoard session where training results can be analysed. TensorBoard also facilitates different tools such as displaying image data (even their features in specific layers)! More information on TensorBoard can be found in <sup>[13](#myfootnote1)</sup>


---

><sup>[13](#myfootnote1)</sup> https://www.tensorflow.org/tensorboard/image_summaries



In [0]:
%load_ext tensorboard
%tensorboard --logdir {join(base_dir, 'logs')}

---
---

## 8.   Play Around!


Congratulations! You finished the tutorial and fine-tuned a pre-trained Xception CNN to classify roof types of buildings in Sint Maarten! Of course there is still a lot more to discover about image recognition so try to play around a bit and use TensorBoard to see what the impact is on the training results.

Some suggestions to adjust in the tutorial:

*   Try 'vgg16' or 'inception' as the pre-trained CNN used in transfer learning
*   Try adjusting the node size of the newly built classification layers in your CNN
*   Try adding an extra classification layer when building the model
*   Try training the model for more epochs
*   Try training the model for more steps_per_epoch
*   Try training the model with a different optimizer
*   Try adjusting the data augmentation rules of the ImageDataGenerators
*   Try adding class weights to the fit_generator method to 'pay more attention' to samples from an under-represented class

Enjoy!





---
---

## 9.   Challenge!


You're now ready to start the challenge and create your own CNN algorithm to classify the roof materials of buildings in Sint Maarten! Use this tutorial as a starting point and try to be creative. Pipple and 510, The Netherlands Red Cross data team, are very curious of the results.

The challange notebook can be opened in the same way as this tutorial is opened, except for the fact that you should now choose "Pydata2019_Challenge_Materials.ipynb" in "Path". For more information you can have a look at today's sent email!